# Importing libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
import gensim
import pickle
import random
import itertools

# load data

In [153]:
data=pd.read_csv('news_articles.csv')['Content']

In [4]:
data.head(),len(data)

(0    At least 14 people died and 17 others were inj...
 1    TV actress Pratibha Tiwari  who is best known ...
 2    The United States and South Korea began a join...
 3    The relentless drive by Bengaluru s  Bangalore...
 4    Punjab Gau Raksha Dal chief Satish Kumar and h...
 Name: Content, dtype: object, 4831)

# Data cleaning

In [237]:
#removing all the ignored 9282 numbers from corpus 
ignored_numbers=pickle.load(open('ignored_numbers.pkl', 'rb'))
for i in range(len(data)):
    query = data[i].split()
    resultwords  = [word for word in query if word not in ignored_numbers]
    data[i] = ' '.join(resultwords)

In [238]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

cleaned_data=[]
tagged_data=[]
for doc in data:
    tokens = word_tokenize(doc.lower())

    # remove stop words
    stop_words = set(stopwords.words('english'))
    words = [w for w in tokens if not w in stop_words]
    #lemmatisation
    lemmatiser = WordNetLemmatizer()
    for tok in words:
        tok=lemmatiser.lemmatize(tok)
    cleaned_data.append(words)
    #pos tagging
    pos=pos_tag(words)
    tagged_data.append(pos)

In [103]:
#store cleaned files
with open('cleaned_data.pkl', 'wb') as f:
    pickle.dump(cleaned_data,f)
with open('tagged_data.pkl', 'wb') as f:
    pickle.dump(tagged_data,f)    

In [218]:
#load files
cleaned_data=pickle.load(open('cleaned_data.pkl','rb'))

# tfidf function

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer()
def tfidf_scores(d):
    tfidf=vectorizer.fit_transform(d)
    return tfidf

In [239]:
n_data=[]
for i in cleaned_data:
    n_data.append(' '.join(i))
new_data=pd.Series(n_data)    
scores=tfidf_scores(n_data)

# Topic modelling

In [48]:
en_model = gensim.models.KeyedVectors.load_word2vec_format('../wiki.en.vec')

In [222]:
len(t_vocab)

45807

In [240]:
t_vocab=list(vectorizer.vocabulary_.keys())
dim=300
en_ignored=[]
en_w2vectors=np.ones((1,dim))
for i in t_vocab:
    if i in en_model:     
        en_w2vectors=np.concatenate((en_w2vectors,en_model[i].reshape(1,-1)),axis=0)
    else:
        en_ignored.append(i)
        en_w2vectors=np.concatenate((en_w2vectors,np.zeros((1,dim))),axis=0)

ignored_numbers = [s for s in en_ignored if s.isdigit()]
len(ignored_numbers)
pickle.dump(ignored_numbers,open('ignored_numbers.pkl', 'wb'))

In [235]:
len(en_ignored)

12690

In [241]:
w2v=en_w2vectors[1:,:]
w2v.shape

(36796, 300)

In [242]:
pickle.dump(w2v,open('w2v.pickle','wb'))

In [243]:
d2v=scores*w2v

In [244]:
pickle.dump(d2v,open('d2v.pickle','wb'))

In [245]:
d2v.shape

(4831, 300)

# clustering

In [24]:
d2v=pickle.load(open('d2v.pickle','rb'))

In [25]:
#from nltk.cluster.kmeans import KMeansClusterer
num_clt=10
from nltk.cluster import KMeansClusterer, cosine_distance
kclusterer = KMeansClusterer(num_clt, distance=cosine_distance, repeats=25)
assigned_clusters = kclusterer.cluster(d2v, assign_clusters=True)

In [26]:
clt_articles={}
for i in range(10):
    clt_articles[i]=np.where(np.array(assigned_clusters)==i) 

In [27]:
pickle.dump(clt_articles,open('cluster.pickle','wb'))


# data generation

In [47]:
# avg. reading speed (200 wpm)
speed=200

In [48]:
l=[]
for i in data:
    l.append(np.array(i.split()).shape[0])
#avg. length of doc (no. of words)
avg_length=np.average(l)

In [49]:
avg_time =avg_length/speed
avg_time*60 #in seconds 

80.391989236183

In [130]:
clt_articles=pickle.load(open('cluster.pickle','rb'))

def generate_data(aid):
    #Generate clickstream data 
    click_stream=np.random.binomial(1,0.75,500)
    
    # Generate time sample from three gaussian distributions
    time = np.random.normal(5,0.5,60)      # just opened and closed
    time = np.append(time, np.random.normal(30,10,160))  # read half/some part   
    time = np.append(time, np.random.normal(80,10,270))   # read completely
    time = np.append(time, np.random.normal(200,10,10))  # app left apened etc.
    #generate 10 values for time
    new_time=time.reshape(-1,1)*click_stream.reshape(-1,1)  
    read_time=random.sample(list(new_time.ravel()),10)
    click=np.ones((len(read_time),1))
    for i in range(len(read_time)):
        if read_time[i]==0:
            click[i]=0
    
    #generating 10 random articles from corpus
    if len(aid)==0:
        aid=[]
        for i in range(10):
            aid.append(random.sample(list(itertools.chain.from_iterable(clt_articles[i])),1)[0])  
    
    #sort aid based on time        
    read_time=np.array(read_time)
    n_liked=len(set(read_time[read_time>50]).intersection(set(read_time[read_time<150])))
    sorted_=[x for _,x in sorted(zip(read_time,aid),reverse=True)]
    liked_=[x for _,x in sorted(zip(read_time,aid),reverse=True)][:n_liked]
    
    #calculating a_rnk and liked articles
    rnk={}
    for j in range(len(sorted_)):
        rnk[sorted_[j]]=j+1    
    a_rnk=[]
    liked=np.zeros((10,1))
    for k in range(len(aid)):
        a_rnk.append(rnk[aid[k]])
        if aid[k] in liked_:
            liked[k]=1
    
    
    return aid,read_time,click,a_rnk,liked


In [131]:
class user():
    id_generator = itertools.count(1)
    def __init__(self,name,article):
        self.name=name
        self.id =next(self.id_generator)
        #self.sid=session
        self.aid,self.time,self.click,self.a_rnk,self.liked=generate_data(article)
    

In [132]:
def user_data(session_id,articles):
    
    if len(articles)!=0:
        user1=user('u1',articles[0])
        user2=user('u2',articles[1])
        user3=user('u3',articles[2])
        user4=user('u4',articles[3])
        user5=user('u5',articles[4])
    else:
        user1=user('u1',[])
        user2=user('u2',[])
        user3=user('u3',[])
        user4=user('u4',[])
        user5=user('u5',[])

    #creating a dataframe for user profiling
    up=pd.DataFrame()
    uid=[]
    aid=[]
    time=[]
    click=[]
    a_rnk=[]
    liked=[]
    
    user_names=[user1,user2,user3,user4,user5]    
    
    for s in range(session_id):
        if s==0:
            for names in user_names:
                uid.append([names.id]*10)
                aid.append(names.aid)
                time.append(names.time)
                click.append(names.click)
                a_rnk.append(names.a_rnk)
                liked.append(names.liked)
                
        else:    
            for n,names in enumerate(user_names): 
                aid_,time_,click_,a_rnk_,liked_=generate_data([])
                uid.append([names.id]*10)
                aid.append(aid_)
                time.append(time_)
                click.append(click_)
                a_rnk.append(a_rnk_)
                liked.append(liked_)
            
    up['uid']=np.array(uid).ravel()
    up['sid']=np.sort(list(range(1,session_id+1))*50)
    up['aid']=np.array(aid).reshape(-1,1)
    up['time']=np.array(time).reshape(-1,1)
    up['click']=np.array(click).reshape(-1,1)
    up['a_rnk']=np.array(a_rnk).reshape(-1,1)
    up['liked']=np.array(liked).reshape(-1,1)

    return up

In [133]:
dummy_data=user_data(4,[])
pickle.dump(dummy_data,open('user_data.pickle','wb'))

# Recommender

In [29]:
# similarity function
from sklearn.metrics.pairwise import cosine_similarity
def similar(d2v,liked,n):
    cos_sim=cosine_similarity(d2v,liked)
    return cos_sim.ravel().argsort()[-(n+1):-1][::-1]

In [170]:
d2v=pickle.load(open('d2v.pickle','rb'))
data=pickle.load(open('user_data.pickle','rb'))
mean_time=data.groupby('aid').mean()['time']

In [128]:
data.head()

,uid,sid,aid,time,click,a_rnk,liked
0,1,1,228,5.563829,1.0,6,0.0
1,1,1,256,68.773441,1.0,2,1.0
2,1,1,606,0.000000,0.0,9,0.0
3,1,1,4633,52.278529,1.0,3,1.0
4,1,1,1326,96.003458,1.0,1,1.0


In [214]:
def new(data,d2v):
    
    #average vector of every user /user profile
    profile=[]
    for j in range(5):
        temp=[]
        for i in data.loc[(data['uid'] ==j+1) & data['liked']==1]['aid'] :    
            weight=int((data[data['aid']==i]['time']/mean_time[i]).values[0])
            temp.append(d2v[i]*weight)  
        profile.append(np.mean(temp,axis=0)) 

    #based on all users (2)
    all_avg=np.mean(profile,axis=0)  #average vector from all users
    trending=similar(d2v,all_avg.reshape(1,-1),2)
    

    #based on user's history (6)
    history=[]
    for vector in profile:
        history.append(similar(d2v,vector.reshape(1,-1),6))
    

    #based on similar users (2)
    sim_users=np.ones((5,5))
    for i in range(len(profile)):
        for j in range(len(profile)):
            sim_users[i][j]=cosine_similarity(profile[i].reshape(1,-1),profile[j].reshape(1,-1))
    sim_users=sim_users.argsort()[:,-3:-1] 
    #print(sim_users)

    #articles based on similar users 
    sim_art=[]
    for i,j in sim_users:
        temp=[]
        temp.append(profile[i])
        temp.append(profile[j])
        sim_art.append(similar(d2v,np.mean(np.array(temp),axis=0).reshape(1,-1),2))
    #return history,sim_art,trending
    new_aid=[]
    for i in range(5):
        temp=[]
        temp.extend(history[i])
        temp.extend(sim_art[i])
        temp.extend(trending)

        new_aid.append(temp)

    return new_aid

In [215]:
new_articles=new(data,d2v)

In [216]:
def show_new(new_articles):
    d=pd.DataFrame(columns=['uid','sid','aid','time','click','a_rnk','liked'])
    uid=[]
    sid=[]
    aid=[]
    time=[]
    click=[]
    a_rnk=[]
    liked=[]
    for n in range(5):
        uid.append([n+1]*10)
        sid.append([6]*10)
        aid.append(new_articles[n])
        time.append(['-']*10)
        click.append(['-']*10)
        a_rnk.append(list(range(1,11)))
        liked.append(['-']*10)
    d['uid']=np.array(uid).ravel()
    d['sid']=np.array(sid).ravel()
    d['aid']=np.array(aid).ravel()
    d['time']=np.array(time).ravel()
    d['click']=np.array(click).ravel()
    d['a_rnk']=np.array(a_rnk).ravel()
    d['liked']=np.array(liked).ravel()
    return d

In [217]:
show_new(new_articles).head()

,uid,sid,aid,time,click,a_rnk,liked
0,1,6,1474,-,-,1,-
1,1,6,898,-,-,2,-
2,1,6,1456,-,-,3,-
3,1,6,1008,-,-,4,-
4,1,6,4822,-,-,5,-
